In [2]:
import pandas as pd

In [3]:
pwd

'C:\\Users\\adhit\\Desktop\\pers\\coding challenge\\git_code\\topic_modelling'

In [4]:
#reading in the data

project_df = pd.read_csv('C:\\Users\\adhit\\Downloads\\project-csv.csv')

In [5]:
project_df.shape

(35303, 19)

# Checking & removing the duplicates


In [6]:
#Removing duplicates

project_df['dup'] = project_df.duplicated(subset=None, keep='first')

In [7]:
project_df.head()

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,ecSignatureDate,frameworkProgramme,masterCall,subCall,fundingScheme,nature,objective,contentUpdateDate,rcn,dup
0,101034345,DOROTHY,SIGNED,DevelOp interdisciplinaRy apprOaches to healTH...,2021-09-01T00:00:00Z,2026-08-31T00:00:00Z,5517000.0,2758500.00,H2020-EU.1.3.,MSCA-COFUND-2020,2021-04-27T00:00:00Z,H2020,H2020-MSCA-COFUND-2020,H2020-MSCA-COFUND-2020,MSCA-COFUND-FP,NaN,Responding to the need to equip European socie...,2021-09-02T20:45:37Z,235801,False
1,101034926,CNergy,SIGNED,Carbon Nanomembranes for Energy Storage Applic...,2021-05-01T00:00:00Z,2022-01-31T00:00:00Z,0.0,100000.00,NaN,FETOPEN-03-2018-2019-2020,NaN,H2020,NaN,H2020-FETOPEN-2018-2019-2020-4,CSA-LSP,NaN,CNM Technologies (CNMT) has developed a highly...,2021-05-21T11:29:04Z,235780,False
2,957151,SATELLITE IA,SIGNED,Advanced MMICs for SOTM and satellites constel...,2020-10-01T00:00:00Z,2021-12-31T00:00:00Z,0.0,109500.00,H2020-EU.2.3.,INNOSUP-02-2019-2020,2020-09-15T00:00:00Z,H2020,H2020-INNOSUP-2018-2020,H2020-INNOSUP-2020-02,CSA-LSP,NaN,The proliferation of LEO (Low Earth Orbit) and...,2021-10-05T18:36:06Z,231593,False
3,970958,TilePlus,SIGNED,Ground-breaking roof tile solution for solar e...,2021-06-01T00:00:00Z,2023-05-31T00:00:00Z,3472500.0,2430750.00,NaN,EIC-FTI-2018-2020,NaN,H2020,NaN,H2020-EIC-FTI-2018-2020,IA,NaN,The overall objective of the project is to mat...,2021-05-21T14:25:08Z,236314,False
4,970564,IMPOWER2X,SIGNED,INERATEC Modular Plants for Renewable Chemical...,2021-05-01T00:00:00Z,2023-04-30T00:00:00Z,3492082.5,2444457.75,NaN,EIC-SMEInst-2018-2020,NaN,H2020,NaN,H2020-EIC-SMEInst-2018-2020-3,SME-2b,NaN,INERATEC (IC) sells modular chemical plants fo...,2021-05-21T14:24:55Z,236312,False


In [9]:
# removing the duplicate columns.

project_df = project_df[project_df['dup'] == False]

In [10]:
project_df.shape

(35303, 20)

In [11]:
#deleting the column 'dup'
del project_df['dup']


# Text Preprocessing

In [12]:
import spacy
nlp = spacy.load('en_core_web_sm')
text = project_df['objective']
